In [1]:
import pandas as pd

In [3]:
pip install requests

In [5]:
pip install beautifulsoup4 

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install selenium

In [2]:
import requests
from bs4 import BeautifulSoup

from urllib.request import urlopen

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
tables = soup.find_all("table")

table = tables[0]
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
df.columns=df.iloc[0,:]
df.drop(index=0,inplace=True)
df.columns

Index(['Postal Code\n', 'Borough\n', 'Neighborhood\n'], dtype='object', name=0)

In [3]:
df = df.replace('\n','', regex=True)

In [4]:
df.rename(columns={"Postal Code\n":'Postal Code' ,"Borough\n":"Borough","Neighborhood\n":'Neighborhood',},inplace=True)

In [5]:
df=df[df.Borough!='Not assigned']

In [6]:
df['Neighborhood'].loc[48:113]

48       Bathurst Manor, Wilson Heights, Downsview North
49                                      Thorncliffe Park
50                              Richmond, Adelaide, King
51                          Dufferin, Dovercourt Village
55                                   Scarborough Village
56                          Fairview, Henry Farm, Oriole
57                       Northwood Park, York University
58         East Toronto, Broadview North (Old East York)
59     Harbourfront East, Union Station, Toronto Islands
60                              Little Portugal, Trinity
64           Kennedy Park, Ionview, East Birchmount Park
65                                       Bayview Village
66                                             Downsview
67                          The Danforth West, Riverdale
68              Toronto Dominion Centre, Design Exchange
69          Brockton, Parkdale Village, Exhibition Place
73                       Golden Mile, Clairlea, Oakridge
74                             

In [15]:
df.shape

(103, 3)

In [7]:
df2=pd.read_csv("C:\\Users\\suvas\\Desktop\\projects\\Geospatial_Coordinates.csv")

In [8]:
df2[df2['Postal Code']== 'M3A']

,Postal Code,Latitude,Longitude
25,M3A,43.753259,-79.329656


In [10]:

df_merge_col = pd.merge(df2, df, on='Postal Code')

In [11]:
df_merge_col[['Postal Code','Borough','Neighborhood','Latitude','Longitude']]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [12]:
df_merge_col['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [13]:
df_merge_col.columns

Index(['Postal Code', 'Latitude', 'Longitude', 'Borough', 'Neighborhood'], dtype='object')

In [14]:
from geopy.geocoders import Nominatim

In [15]:
CLIENT_ID = '5PUZADNNWEHF2X4QZQNFHUBBGGETMHNYAA5UOMALUJP0TS01' # your Foursquare ID
CLIENT_SECRET = 'ITIGXCFBI1AQ1VIRN4QSU1GR4L3SJZYI12LPSDAWUUMEVG3F' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5PUZADNNWEHF2X4QZQNFHUBBGGETMHNYAA5UOMALUJP0TS01
CLIENT_SECRET:ITIGXCFBI1AQ1VIRN4QSU1GR4L3SJZYI12LPSDAWUUMEVG3F


In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [17]:
import folium
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Toronto

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


for lat, lng, borough,neighbourhood in zip(df_merge_col.Latitude, df_merge_col.Longitude, df_merge_col.Borough,df_merge_col.Neighborhood):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(venues_map) 
venues_map




In [18]:
scarborough_data= df_merge_col[df_merge_col.Borough=='Scarborough']
scarborough_data.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [19]:
address = 'Scarborough'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude_Scarborough = location.latitude
longitude_Scarborough = location.longitude
print(latitude_Scarborough, longitude_Scarborough)

54.2820009 -0.4011868


In [20]:
scarborough_data= df_merge_col[df_merge_col.Borough=='Scarborough']
scarborough_data.head()
scarborough_data.drop(columns=['Borough'],axis=0, inplace=True)

locations=scarborough_data[['Latitude','Longitude']]

locationslist= locations.values.tolist()

locationslist
map = folium.Map(location=[54.2820009, -0.4011868], zoom_start=12)
for point in range(0, len(locationslist)):
    folium.Marker(locationslist[point], popup=scarborough_data['Neighborhood'][point]).add_to(map)
map


c:\users\suvas\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
